# Lecture 2a: Eigendecomposition of Data and Systems
In this lecture, we will discuss the eigendecomposition of a square matrix and how it can be used to understand data and systems in unsupervised machine learning. There are several key ideas in this lecture:

* __Eigendecomposition__ allows us to decompose a matrix into its constituent parts, the [eigenvectors and eigenvalues](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors). These values help us understand the structure of the data or system represented by the matrix. We'll look at two approaches to estimate the [eigenvalues and eigenvectors](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors) of a matrix:
    * __Power iteration method__ estimates the _largest_ eigenvalue/eigenvector pair. Given a _diagonalizable_ matrix $\mathbf{A}$ the power iteration algorithm will produce a number $\lambda$, which is the greatest (in absolute value) eigenvalue of $\mathbf{A}$ and a nonzero vector $\mathbf{v}$ which is a corresponding eigenvector of $\lambda$ such that $\mathbf{A}\mathbf{v} = \lambda\cdot\mathbf{v}$.
    * __QR iteration__ is another approach to compute the eigendecomposition of the matrix $\mathbf{A}$. However, unlike power iteration, this approach will give all eigenvalues and eigenvectors of the matrix $\mathbf{A}$. The QR factorization algorithm relies on the [QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition), which itself relies on the [Gram-Schmidt algorithm](https://en.wikipedia.org/wiki/Gram–Schmidt_process).
* __Buy versus build__: While we will explore these two approaches to compute the eigendecomposition of a matrix in the lecture and associated lab, most, if not all, computing platforms already have built-in functionality to do this computation. For example, [Julia has the `eigen(...)` function exported by the LinearAlgebra.jl package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen). Most of the time, there is no need to recreate the wheel (and your implementation will likely be worse in space and time complexity). So buy!

Lecture notes can be found: [here!](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-2/L2a/docs/Notes.pdf)

## Setup, Data and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem.

In [3]:
include("Include.jl");

We'll use the coagulation dataset. Let's load this data from disk using [the `MySyntheticDataSet()` function](src/Files.jl)

In [5]:
dataset = MySyntheticDataset() |> d-> d["ensemble"]; 

The keys of the dataset dictionary are the `actual` patient indexes. These keys point to `synthetic` patient measurement vectors constructed by building a model of the original data distribution. To explore this data, specify an original patient index (one of the keys of the original dictionary) in the `original_patient_index::Int` variable:

In [7]:
original_patient_index = 7; # i ∈ {keys}

Next, we'll build a data matrix with the `synthetic` measurement vectors for the specified original patient index. We'll store this in the `D::Array{<:Number, 1}` matrix. 

In [9]:
D = let

    M = dataset[original_patient_index];
    number_of_rows = length(M); # number of synthetic patients
    number_of_cols = length(M[1]) - 1; # number of measurements (features), first col is the visit number
    D = Array{Float64,2}(undef, number_of_rows, number_of_cols);

    for i ∈ 0:(number_of_rows - 1)
        for j ∈ 1:(number_of_cols)
            D[i+1,j] = M[i][j+1];
        end
    end

    # D̂ = copy(D); # no z-scale this data
    # for j ∈ 1:number_of_cols
    #     sample_vector = D[:,j]; 
    #     μ = mean(sample_vector);
    #     σ = std(sample_vector);

    #     for i ∈ 1:number_of_rows
    #         D̂[i,j] = (sample_vector[i] - μ)/σ;
    #     end
    # end
    
    D
end;

Finally, we set some constants that we'll use throughout the lecture. See the comment beside the constant value for its meaning, permissible values, units, etc.

In [11]:
number_of_examples = size(D,1); # number of synthetic patients
number_of_features = size(D,2); # number of features (measurements)
maxiter = 100; # maximum number of iterations
ϵ = 1e-8; # stopping criteria

## Eigendecomposition
Suppose we have a real square matrix $\mathbf{A}\in\mathbb{R}^{m\times{m}}$ which could be a measurement dataset, e.g., the columns of $\mathbf{A}$ represent feature 
vectors $\mathbf{x}_{1},\dots,\mathbf{x}_{m}$ or an incidence array in a graph with $m$ nodes, etc. Eigenvalue-eigenvector problems involve finding a set of scalar values $\left\{\lambda_{1},\dots,\lambda_{m}\right\}$ called 
[eigenvalues](https://mathworld.wolfram.com/Eigenvalue.html) and a set of linearly independent vectors 
$\left\{\mathbf{v}_{1},\dots,\mathbf{v}_{m}\right\}$ called [eigenvectors](https://mathworld.wolfram.com/Eigenvector.html) such that:
$$
\begin{equation}
\mathbf{A}\cdot\mathbf{v}_{j} = \lambda_{j}\cdot\mathbf{v}_{j}\qquad{j=1,2,\dots,m}
\end{equation}
$$
where $\mathbf{v}\in\mathbb{C}^{m}$ and $\lambda\in\mathbb{C}$. So, why is this interesting?
* Eigenvectors represent fundamental directions of the matrix $\mathbf{A}$. For the linear transformation defined by a matrix $\mathbf{A}$, eigenvectors are the only vectors that do not change direction during the transformation. If we think about the matrix $\mathbf{A}$ as a machine, we put the eigenvector $\mathbf{v}_{\star}$ into the machine, and we get back the same eigenvector $\mathbf{v}_{\star}$ multiplied by a scalar, the eigenvalue $\lambda_{\star}$.
* Eigenvalues are scale factors for their eigenvector. An eigenvalue is a scalar that indicates how much a corresponding eigenvector is stretched or compressed during a linear transformation represented by the matrix $\mathbf{A}$.

Another interpretation we'll explore later is that eigenvectors represent the most critical directions in the data or system, and the eigenvalues represent the importance of these directions. Hmmm, interesting. But how can we calculate them (given the buy versus build caveat above)?

### Method 1: Power iteration
The [power iteration method](https://en.wikipedia.org/wiki/Power_iteration) is an iterative algorithm to compute the largest eigenvalue and its corresponding eigenvector of a square (real) matrix; we'll consider only real-valued matrices here, but this approach can also be used for matrices with complex entries. 

__Phase 1: Eigenvector__: Suppose we have a real-valued square _diagonalizable_ matrix $\mathbf{A}\in\mathbb{R}^{m\times{m}}$ whose eigenvalues have the property $|\lambda_{1}|\geq|\lambda_{2}|\dots\geq|\lambda_{m}|$. Then, the eigenvector $\mathbf{v}_{1}\in\mathbb{C}^{m}$ which corresponds to the largest eigenvalue $\lambda_{1}\in\mathbb{C}$ can be (iteratively) estimated as:
$$
\mathbf{v}_{1}^{(k+1)} = \frac{\mathbf{A}\mathbf{v}_{1}^{(k)}}{\Vert \mathbf{A}\mathbf{v}_{1}^{(k)} \Vert}\quad{k=0,1,2\dots}
$$

where $\lVert \star \rVert$ denotes the [L2 (Euclidean) vector norm](https://mathworld.wolfram.com/L2-Norm.html). The [power iteration method](https://en.wikipedia.org/wiki/Power_iteration) converges to a value for the eigenvector as $k\rightarrow\infty$ when a few properties are true, namely, $|\lambda_{1}|/|\lambda_{2}| < 1$ (which is unknown beforehand), and we pick an appropriate initial guess for $\mathbf{v}_{1}$ (in our case, a random vector will work).

__Phase 2: Eigenvalue__: Once we have an estimate for the eigenvector $\hat{\mathbf{v}}_{1}$, we can estimate the corresponding eigenvalue $\hat{\lambda}_{1}$ using [the Rayleigh quotient](https://en.wikipedia.org/wiki/Rayleigh_quotient). This argument proceeds from the definition of the eigenvalues and eigenvectors. We know, from the definition of eigenvalue-eigenvector pairs, that:
$$
\mathbf{A}\hat{\mathbf{v}}_{1} - \hat{\lambda}_{1}\hat{\mathbf{v}}_{1}\simeq{0}
$$
where we use the $\simeq$ symbol because we don't have the true eigenvector $\mathbf{v}_{1}$, only an estimate of it. To solve this expression for the (estimated) eigenvalue $\hat{\lambda}_{1}$, we multiply through by the transpose of the eigenvector and solve for the eigenvalue:
$$
\hat{\lambda}_{1} \simeq \frac{\hat{\mathbf{v}}_{1}^{T}\mathbf{A}\hat{\mathbf{v}}_{1}}{\hat{\mathbf{v}}_{1}^{T}\hat{\mathbf{v}}_{1}} = \frac{\left<\mathbf{A}\hat{\mathbf{v}}_{1},\hat{\mathbf{v}}_{1}\right>}{\left<\hat{\mathbf{v}}_{1},\hat{\mathbf{v}}_{1}\right>}
$$
where $\left<\star,\star\right>$ denotes [an inner product](https://mathworld.wolfram.com/InnerProduct.html). 

__Algorithm__
* __Initialization__. We begin (iteration $k=0$) with an initial (random) guess of the eigenvector $\mathbf{v}_{1}^{(0)}$, the maximum number of iterations we are willing to take `maxiter,` and a tolerance parameter $\epsilon>0$.  
* __Update__: Next, we repeatedly multiply the $\mathbf{v}^{\star}_{1}$ vector by the matrix $\mathbf{A}$ and normalize the result by $\Vert\mathbf{A}\mathbf{v}^{\star}_{1}\Vert$. This iterative approach capitalizes on the property that the dominant eigenvalue will exert the most influence on the vector $\mathbf{v}$ over successive iterations, allowing it to converge towards the eigenvector associated with the largest eigenvalue.
* __Stopping__: We stop the iteration procedure after `maxiter` number of iterations is reached or when the difference between successive iterations is _small_ in some sense, i.e., $\lVert \mathbf{v}_{1}^{(k)} - \mathbf{v}_{1}^{(k-1)} \rVert\leq\epsilon$ where $\lVert\star\rVert$ is [some vector norm](https://mathworld.wolfram.com/VectorNorm.html). In practice, we'll use both stopping criteria and the L2 norm to guard against an infinite loop (our iteration will be implemented using [a `while` loop](https://docs.julialang.org/en/v1/base/base/#while)).

While simple to implement, the [power iteration method](https://en.wikipedia.org/wiki/Power_iteration) may exhibit slow convergence, mainly when the largest eigenvalue is close in magnitude to other eigenvalues, i.e., $|\lambda_{1}|/|\lambda_{2}| \sim 1$.
Check out a [power iteration pseudo-code here!](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-1/L1c/figs/pcode-kmeans.pdf)

Additional references:
* https://www.cs.cornell.edu/~bindel/class/cs6210-f16/lec/2016-10-17.pdf
* https://blogs.sas.com/content/iml/2012/05/09/the-power-method.html

#### Implementation
We've implemented the [power iteration method](https://en.wikipedia.org/wiki/Power_iteration) in the [`poweriteration(...)` function](src/Compute.jl). 
* The [`power iteration (...)` function](src/Compute.jl) takes the square matrix $\mathbf{A}$, an initial guess for the eigenvector $\mathbf{v}^{(0)}_{1}$ and (optional) keyword parameters controlling the stopping criteria as arguments. The function returns a tuple holding the estimated eigenvector $\hat{\mathbf{v}}_{1}$ and eigenvalue $\hat{\lambda}_{1}$.

In [15]:
(v̂,λ̂) = let

    A = transpose(D)*D; # build a square matrix from the data D^T*D gives measure x measure, D*D^T gives patient x patient 
    n = size(A,1); # how many rows (cols) do we have? (square)
    vₒ = randn(n); # initial random guess

    # call the poweriteration function
    (v, λ) = poweriteration(A, vₒ, maxiter = maxiter, ϵ = ϵ);

    # return -
    (v,λ)
end;

Converged in 4 iterations


#### Test
To test our implementation, let's compare the values of $(\hat{\lambda}_{1}, \hat{\mathbf{v}}_{1})$ that we estimated with those computed [using the `eigen(...)` function exported by the LinearAlgebra.jl package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen). 
* The built-in [`eigen(...)` function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen) takes the matrix $\mathbf{A}$ (and some additional optional arguments) and returns [an `Eigen` factorization object](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.Eigen) holding the eigenvalues and eigenvectors.

In [17]:
let
    A = transpose(D)*D; # build a square matrix from the data
    F = eigen(A); # compute the eigendecomposition

    # get
    λ = maximum(F.values); # get the max eigenvalue (sorted, this should be the last element)
    v = F.vectors[:,end]; # eigenvectors are sorted - the last column is v₁

    # tests
    @assert (λ ≈ λ̂) && (abs.(v) ≈ abs.(v̂))  # do the eigenvalues and eigenvectors match?
end

### Method 2: QR Iteration
[QR iteration](https://en.wikipedia.org/wiki/QR_algorithm) is a fundamental technique in numerical linear algebra, primarily used for computing the eigenvalues and eigenvectors of matrices. The algorithm leverages the concept of [QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition), which expresses a (rectangular) matrix $\mathbf{A}\in\mathbb{R}^{n\times{m}}$ as a product of an orthogonal matrix $\mathbf{Q}\in\mathbb{R}^{n\times{n}}$ and an upper triangular matrix $\mathbf{R}\in\mathbb{R}^{n\times{m}}$:
$$
\mathbf{A} = \mathbf{Q}\mathbf{R}
$$
where $\mathbf{Q}^{T}\mathbf{Q} = \mathbf{I}$. The core of the QR iteration algorithm involves iteratively decomposing a given matrix $\mathbf{A}$ into its $\mathbf{Q}$ and $\mathbf{R}$ factors and then reformulating the matrix for subsequent iterations. 

__Algorithm__
* __Initialization__. We begin by specifying an initial matrix $\mathbf{A}_{1} = \mathbf{A}$, the maximum number of iterations `maxiter` that we are willing to do, and a tolerance parameter $\epsilon$.
* __Update__. For iteration $k = 1,2,\dots$, compute the [QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition) of $\mathbf{A}_{k} = \mathbf{Q}_{k}\mathbf{R}_{k}$. We then form a new matrix $\mathbf{A}_{k+1} = \mathbf{R}_{k}\mathbf{Q}_{k}$, which can be re-written as $\mathbf{A}_{k+1} = \mathbf{Q}^{T}_{k}\mathbf{A}_{k}\mathbf{Q}_{k}$.
* __Stopping__. We stop the iteration procedure after `maxiter` number of iterations is reached or when the difference between successive iterations is _small_ in some sense, i.e., $\lVert \mathbf{A}_{k+1} - \mathbf{A}_{k} \rVert\leq\epsilon$ where $\lVert\star\rVert$ is a [Matrix norm](https://en.wikipedia.org/wiki/Matrix_norm), e.g., the $p = 1$ norm and $\epsilon$ is a tolerance parameter.

Under certain conditions, [QR iteration](https://en.wikipedia.org/wiki/QR_algorithm) will converge to a triangular matrix with the eigenvalues of the original matrix $\mathbf{A}$ listed on the diagonal. To compute the eigenvectors, we solve the homogenous system of linear algebraic equations:
$$
\left(\mathbf{A} - \lambda_{\star}\mathbf{I} \right)\cdot\mathbf{v}_{\star} = \mathbf{0}
$$

However, before we implement [QR iteration](https://en.wikipedia.org/wiki/QR_algorithm), let's look at how to compute the $\mathbf{Q}$ and $\mathbf{R}$ matricies.

#### Aside: Classical and Modified Gram-Schmidt
The [QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition) can be computed using a variety of approaches, including a handy technique called [the Gram–Schmidt algorithm](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process). Gram-Schmidt computes a set of two or more vectors that are orthogonal (perpendicular) to each other. The vector projection operation is the basic unit operation of [the Gram–Schmidt algorithm](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process). The projection of vector $\mathbf{v}$ on a nonzero vector $\mathbf{u}$ is given by: 
$$
P_{\mathbf{u}}(\mathbf{v}) = \frac{\left<\mathbf{v},\mathbf{u}\right>}{\left<\mathbf{u},\mathbf{u}\right>}\cdot\mathbf{u}
$$
where $\left<\star,\star\right>$ denotes [an inner product](https://mathworld.wolfram.com/InnerProduct.html).  Classical Gram-Schmidt allows us to compute a set of orthogonal vectors using the projection operation shown above, i.e., given vectors $\mathbf{v}_{1},\mathbf{v}_{2},\dots,\mathbf{v}_{k}$ the classical Gram–Schmidt process defines the vectors $\mathbf{u}_{1},\mathbf{u}_{2},\dots,\mathbf{u}_{k}$ as:

$$
\begin{align*}
\mathbf{u}_{1} & = \mathbf{v}_{1} \\
\mathbf{u}_{2} & = \mathbf{v}_{2} - P_{\mathbf{u}_{1}}(\mathbf{v}_{2})\\
\mathbf{u}_{3} & = \mathbf{v}_{3} - P_{\mathbf{u}_{1}}(\mathbf{v}_{2}) - P_{\mathbf{u}_{2}}(\mathbf{v}_{3})\\
\vdots & =  \vdots \\
\mathbf{u}_{k} & = \mathbf{v}_{k} - \sum_{j=1}^{k-1}P_{\mathbf{u}_{j}}(\mathbf{v}_{k})
\end{align*}
$$
Classical Gram-Schmidt can sometimes produce _almost_ orthogonal vectors because of roundoff error. Let's look at a classic Gram-Schmidt example before we fix this issue and compute our $\mathbf{Q}$ and $\mathbf{R}$.

In [39]:
(A, Q) = let
    A = randn(10,3);
    Q = orthogonalize(A, ClassicalGramSchmidtAlgorithm())
    A,Q
end;

Once we have $\mathbf{Q}$, we can compute the $\mathbf{R}$ factor as:
$$
\mathbf{R} = \mathbf{Q}^{T}\mathbf{A}
$$

In [22]:
R = transpose(Q)*A;

In [23]:
A

10×3 Matrix{Float64}:
 -0.11095     0.0562906  -0.293299
 -2.42952     0.600019    0.0394207
  0.370953   -1.93396     0.510159
  1.16786    -1.22952     1.04411
 -1.76285     0.175377   -0.981568
 -1.64329     1.11924    -3.0703
 -0.954255    0.140181   -0.916317
  0.0939819  -1.07469     0.621494
  0.524992    0.199492   -0.495168
  0.86224    -0.491428   -1.06302

In [24]:
Q*R

10×3 Matrix{Float64}:
 -0.11095     0.0562906  -0.293299
 -2.42952     0.600019    0.0394207
  0.370953   -1.93396     0.510159
  1.16786    -1.22952     1.04411
 -1.76285     0.175377   -0.981568
 -1.64329     1.11924    -3.0703
 -0.954255    0.140181   -0.916317
  0.0939819  -1.07469     0.621494
  0.524992    0.199492   -0.495168
  0.86224    -0.491428   -1.06302